In [1]:
# 02_AQI_Point_Maps_LocalGuideline.ipynb

import os, glob, warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.ops import unary_union
from fiona.env import Env

# ---- Paths
DATA_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150

# ---- Column helpers
PM_CANDS  = ["pm25","PM25","PM_25","PM2_5","PM2.5"]
LON_CANDS = ["longitude","lon","LONG","x"]
LAT_CANDS = ["latitude","lat","LAT","y"]

def pick(cols, cands):
    for c in cands:
        if c in cols:
            return c
    return None

# ---- Local PM2.5 guideline (µg/m³, 24-hr)
# Good: 0–25.0
# Fair: 25.1–35.0
# Unhealthy for sensitive groups: 35.1–45.0
# Very unhealthy: 45.1–55.0
# Acutely unhealthy: 55.1–90.0
# Emergency: >= 91.0
NO_DATA_LABEL = "No data"
NO_DATA_COLOR = "#bdbdbd"

AQI_LABELS = [
    "Good (0–25.0)",
    "Fair (25.1–35.0)",
    "Unhealthy for sensitive (35.1–45.0)",
    "Very unhealthy (45.1–55.0)",
    "Acutely unhealthy (55.1–90.0)",
    "Emergency (≥91)"
]
AQI_COLORS = [
    "#00E400",  # Good
    "#FFFF00",  # Fair
    "#FF7E00",  # Unhealthy SG
    "#FF0000",  # Very unhealthy
    "#8F3F97",  # Acutely unhealthy
    "#7E0023"   # Emergency
]

def classify_pm25(val):
    """Return (label, color) per local guideline; robust to NaN/strings."""
    v = pd.to_numeric(val, errors="coerce")
    if not np.isfinite(v):
        return NO_DATA_LABEL, NO_DATA_COLOR
    v = max(0.0, float(v))  # no negatives
    if v <= 25.0:   return AQI_LABELS[0], AQI_COLORS[0]
    elif v <= 35.0: return AQI_LABELS[1], AQI_COLORS[1]
    elif v <= 45.0: return AQI_LABELS[2], AQI_COLORS[2]
    elif v <= 55.0: return AQI_LABELS[3], AQI_COLORS[3]
    elif v <  91.0: return AQI_LABELS[4], AQI_COLORS[4]   # 55.1–90.999…
    else:           return AQI_LABELS[5], AQI_COLORS[5]   # ≥ 91

# ---- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# ---- Iterate daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    date_label = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)

    # Ensure geometry exists (build from lon/lat if necessary)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = pick(g.columns, LON_CANDS)
        lat = pick(g.columns, LAT_CANDS)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print(f"Skip (no geometry & no lon/lat): {base}")
            continue

    pm_col = pick(g.columns, PM_CANDS)
    if pm_col is None:
        print(f"Skip (no PM2.5 column): {base}")
        continue

    # Reproject & classify per guideline
    pts = g.to_crs(UTM51).copy()
    classes = pts[pm_col].apply(classify_pm25)
    pts["AQI_LABEL"] = [c[0] for c in classes]
    pts["AQI_COLOR"] = [c[1] for c in classes]

    # ---- Plot
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)
    pts.plot(ax=ax,
             color=pts["AQI_COLOR"],
             markersize=60,
             edgecolor="white", linewidth=0.8)

    ax.set_title(f"Stations by PM₂.₅ Category — {date_label}")
    ax.set_axis_off()

    # Legend (include 'No data' only if present)
    patches = [mpatches.Patch(color=c, label=l) for l, c in zip(AQI_LABELS, AQI_COLORS)]
    if (pts["AQI_LABEL"] == NO_DATA_LABEL).any():
        patches.append(mpatches.Patch(color=NO_DATA_COLOR, label=NO_DATA_LABEL))
    ax.legend(handles=patches, loc="lower left", bbox_to_anchor=(0.01,0.01),
              frameon=True, fontsize=9)

    out = os.path.join(OUT_DIR, f"{base}_aqi_points_local.png")
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out)

Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-06_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-07_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-08_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-09_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-10_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-11_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-12_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-13_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-14_aqi_points_lo